In [ ]:
import numpy as np
import pandas as pd#
import matplotlib.pyplot as plt
import glob
import matplotlib as mpl
from cycler import cycler
from itertools import filterfalse

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes 
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [ ]:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.sans-serif": "Helvetica",
# })

# NON INTERP MODELS

In [ ]:
cols = np.loadtxt("boost_models/fdat-columns.txt", dtype=str)

In [ ]:
cols_new = [c.split(":")[1] for c in cols]

In [ ]:
files = glob.glob("boost_models/f*.d*")

In [ ]:
linestyles = ['solid',(0, (3, 1, 1, 1, 1, 1)),'dotted','dashed','dashdot']

In [ ]:
cols = np.loadtxt("boost_models/fdat-columns.txt", dtype=str)
cols_new = [c.split(":")[1] for c in cols]
files = glob.glob("boost_models/f*.d*")
linestyles = ['solid',(0, (3, 1, 1, 1, 1, 1)),'dotted','dashed','dashdot']
plt.figure(figsize=(7,6))
for file, ls in zip(files,linestyles):
# for file in files:
    lab=file.split("/")[1].split("-")[0][1:]
    df = pd.read_csv(file, sep=" ", names=cols_new)
    
    a = plt.plot(np.log10(df["Teff[K]"]), df["log(L/Lsun)"], label=str(int(lab)), ls=ls, c="k",
                lw=0.8)
    grav_to_scatter = (df["logg"] > 3.42) & (df["logg"] < 4.42)
#     print(lab, df["logg"].max())
    gs = plt.scatter(np.log10(df["Teff[K]"])[grav_to_scatter], df["log(L/Lsun)"][grav_to_scatter],
                 c=df["logg"][grav_to_scatter], s=10, cmap="summer")
cb = plt.colorbar()
cb.set_label(label=r"$\log(g/{\rm cm\ s}^{-2})$", size=15)
cb.ax.tick_params(labelsize=13)
plt.gca().invert_xaxis()
plt.gca().text(0.95,0.05, r"$Z/Z_{\odot} \sim 1/25$", transform=plt.gca().transAxes,
              va="bottom", ha="right",size=15)
plt.gca().tick_params(which="major", direction="inout",size=5, top=True, right=True, labelsize=13)

plt.xlabel(r"$\log(T_{\rm eff}/{\rm K})$", size=15)
plt.ylabel(r"$\log(L/L_{\odot})$", size=15)
leg = plt.legend(title=r"$M/M_{\odot}$")
plt.setp(plt.gca().get_legend().get_title(), fontsize='11')
plt.setp(plt.gca().get_legend().get_texts(), fontsize='10')

plt.show()

# INTERP MODELS

In [ ]:
filename = "boost_models/interpolatedtracks-dwarfB.dat"

In [ ]:
with open(filename, 'r') as fobj:
    masses = [] # mass of each model
    interped_dfs = [] # this will be corresponding data.
    rows_of_current_model = [] # list to store each row of a model
    for i, line in enumerate(fobj): # loop through file
        # take column headers as first line, from 1:-1 to not take
        # the "#" and the "\n".
        if i == 0:
            first_filter = list(filterfalse(lambda val: val == "",line[1:-1].split(" ")))
            second_filter = list(filterfalse(lambda col: col.startswith("("), first_filter))
            column_headers_interp = [col[:-4] if "(" in col else col for col in second_filter ]
            # print(column_headers_interp)
        elif line.startswith("#"): # if it is a header, store the mass
            if i > 1:
                # Store interpolated model as a df
                # interped_dfs.append(rows_of_current_model)
                interped_dfs.append(rows_of_current_model)
            mass = float(line.split(" ")[4])
            if mass > 50:
                break
            masses.append(mass)
            rows_of_current_model = []
        else:
            if line != "\n": # last two rows are line skips, don't want
                # also, final 2 characters of each line are "\n", dont want.
                # So we need the numerical values for the dataframes,
                # use filterfalse itertool to not use values that equal
                # empty string.
                rows_of_current_model.append(np.array(list(filterfalse(lambda val: val == "", line[:-2].split(" "))), dtype=float))


In [ ]:
masses = np.array(masses)
interped_dfs = np.array(interped_dfs)

In [ ]:
mdot_models = pd.read_csv("mdot_model_params.txt", delim_whitespace=True)
abundances = ["0.01", "0.03", "0.05"]
markers = ["o", "^", "*", "s", "P"]

In [ ]:
def round_to_2(x, dx):
    """Function that returns x to 2 significant digits. 
    Used for determining uncertainties, and for printing
    values.
    
    ----------
    x : float
        Input number   
    dx : float
        x uncertainty.
    
    Returns
    -------
    x_same_as_dx, dx_2sig_figs : tuple of float
        dx to 2 significant figures, and x at the
        same number of decimal points as this.
    
    """
    dx_2sig_figs = "%s" % float("%.2g" % dx)
    digits_after_point = len(dx_2sig_figs.split(".")[1])
    # Now see if, to 2 sigfigs, dx is 2 decimal points, but
    # the final digit is a 0.
    if (str(dx)[0] == "0") and (digits_after_point == 1):
        digits_after_point  += 1
        dx_2sig_figs += "0"
        
    x_same_as_dx = str(round(x,digits_after_point))

    if (len(x_same_as_dx.split(".")[1]) == 1) and (digits_after_point == 2):
        x_same_as_dx += "0"
        
    return x_same_as_dx, dx_2sig_figs

In [ ]:
linestyles = ['dotted','dashed','dashdot']

fig, axs = plt.subplots(1,3,figsize=(18,6))
for i, ax in enumerate(axs):
    # Now plot models of given abundance.
    rows_to_plot = mdot_models[mdot_models["C/Csun"] == float(abundances[i])]
    rows_to_plot = rows_to_plot.set_index(np.arange(5)) # reset indices for iterrows.
    
    # Make the zoomed-in axis.
    axins = zoomed_inset_axes(ax, 2.2, loc=1) # zoom = 25
    
    for j, row in rows_to_plot.iterrows():  
        model_mass = row["M/Msun"]
        dmodel_mass = row["dM/Msun"]
        model_grav = row["log(g)"]
        label = round_to_2(model_mass,dmodel_mass)
        ax.errorbar(np.log10(row["Teff(K)"]), row["log(L/Lsun)"],
                 xerr=row["dTeff(K)"]/(np.log(10)*row["Teff(K)"]),
                 yerr=row["dLogL/Lsun"],
                 elinewidth=0.7, ms=6, label=rf"{label[0]} $\pm$ {label[1]}$M_\odot$",#", {model_grav}",
                 c="k", ls=" ", marker=markers[j])
        axins.errorbar(np.log10(row["Teff(K)"]), row["log(L/Lsun)"],
                 xerr=row["dTeff(K)"]/(np.log(10)*row["Teff(K)"]),
                 yerr=row["dLogL/Lsun"],
                 elinewidth=0.7, ms=6, label=rf"{label[0]} $\pm$ {label[1]}$M_\odot$",#", {model_grav}",
                 c="k", ls=" ", marker=markers[j])
        
        if j >1:
            closest_idx = np.argmin(abs(model_mass-masses))
            closest_model = pd.DataFrame(interped_dfs[closest_idx][:-3],
                                        columns=column_headers_interp)
            closest_mass = masses[closest_idx]
            closest_gravs = np.log10(np.array((6.674e-8 * closest_model["mass"]) / closest_model["Rstar"]**2))
            ax.plot(np.log10(closest_model["Teff"]),
                    np.log10(closest_model["Lbol"]/ 3.846e33),
                    c="k", ls=linestyles[j-2],
                    label=fr"{closest_mass:.2f} $M_\odot$")
            
            axins.plot(np.log10(closest_model["Teff"]),
                    np.log10(closest_model["Lbol"]/ 3.846e33),
                    c="k", ls=linestyles[j-2],
                    label=fr"{closest_mass:.2f} $M_\odot")
                       
    ax.set_title(fr"[C] = {abundances[i]} C$_\odot$", size=16, y=1.02)
    ax.legend()

    xmin, xmax = np.log10(rows_to_plot["Teff(K)"]).min()-0.12, np.log10(rows_to_plot["Teff(K)"]).max()+0.12
    ymin, ymax = rows_to_plot["log(L/Lsun)"].min()-0.15, rows_to_plot["log(L/Lsun)"].max()+0.15
    
    axins.set_xlim(xmin, xmax)
    axins.set_ylim(ymin, ymax)
    axins.invert_xaxis()
                       
    if i == 0:
        ax.set_ylabel(r"$\log(L/L_{\odot})$", size=18)
                       
    ax.set_xlabel(r"log($T_{\rm eff}$/K)", size=18)
    ax.tick_params(which="major", top=True, right=True, direction="inout", length=8,labelsize=15)
    ax.invert_xaxis()
plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(3,3,figsize=(18,15))

for i, row in enumerate(axs):    
    for j, ax in enumerate(row):
        # Find point
        current_row = mdot_models[mdot_models["C/Csun"] == float(abundances[j])].iloc[3]
        # Plot it
        model_mass = current_row["M/Msun"]
        dmodel_mass = current_row["dM/Msun"]
        model_L = current_row["log(L/Lsun)"]
        model_dL = current_row["dLogL/Lsun"]
        model_logT = np.log10(current_row["Teff(K)"])
        model_dlogT = current_row["dTeff(K)"]/(np.log(10)*current_row["Teff(K)"])
        model_grav = current_row["log(g)"]

        label = round_to_2(model_mass, dmodel_mass) # for legend

        mass_range_idxs = (masses < model_mass + dmodel_mass) & (masses > model_mass - dmodel_mass)

        mass_range = masses[mass_range_idxs]
        mass_range_models = interped_dfs[mass_range_idxs]
        mass_range_logL = np.log10(mass_range_models[:,:,6]/ 3.846e33)
        mass_range_logT = np.log10(mass_range_models[:,:,8])
        ax.errorbar(model_logT, model_L, xerr=model_dlogT, yerr=model_dL, 
                    c="k", ls=" ", marker="s",zorder=10, ms=6,
                    label=fr"$M=$ {label[0]} $\pm$ {label[1]} $M_\odot$"+"\n"+r"log($g$/cm s$^{-2}$) = "+str(model_grav))
        
        if i == 0:
        ############ Find gravities within range I tested.
            closest_grav_range = np.log10((6.674e-8 * mass_range_models[:,:,2]) / mass_range_models[:,:,7]**2)
            grav_to_scatter = (closest_grav_range > 3.42) & (closest_grav_range < 4.42)
            gs= ax.scatter(mass_range_logT[grav_to_scatter],
                           mass_range_logL[grav_to_scatter],
                           c=closest_grav_range[grav_to_scatter],
                           s=30, cmap="summer", vmin=3.42, vmax=4.42)
            cb = fig.colorbar(gs, ax=ax)
            cb.ax.tick_params(which="major", labelsize=13)
            if j == 2:
                cb.set_label(r"$\log(g/{\rm cm\ s}^{-2})$", size=17)
                
        elif i == 1:
            ################# CARBON ABUNDANCES
            surf_C11 = mass_range_models[:,:,28]
            surf_C12 = mass_range_models[:,:,29]
            surf_C13 = mass_range_models[:,:,30]
            
            surf_C_tot_mass = surf_C11 + surf_C12 + surf_C13 # total mass of carbon in grams, stellar masses cancel
            surf_C_tot_number = surf_C_tot_mass / 1.9944733e-23
            
            surf_H1 = mass_range_models[:,:,17]
            surf_H2 = mass_range_models[:,:,18]
            
            surf_H_tot_mass = surf_H1 + surf_H2 
            surf_H_tot_number = surf_H_tot_mass / 1.6735575e-24 
            
            c_abundance = np.log10(surf_C_tot_number / surf_H_tot_number) + 12

            c_over_csun = 10**(c_abundance - 8.43) # 8.43 is solar value from asplund.

            cs= ax.scatter(mass_range_logT,
                           mass_range_logL,
                           c=c_over_csun,
                           s=30, cmap="magma",
                           vmin=0.006)#, #vmin = 0.013 for db , 05 for izw
                           #vmax=c_over_csun.max())
            cb = fig.colorbar(cs, ax=ax)
            cb.ax.tick_params(which="major", labelsize=13)
            if j == 2:
                cb.set_label(r"[C] (C$_\odot$)", size=17)
                
        elif i == 2:
            ############# VROT
            vrot = mass_range_models[:,:,11]
            vs= ax.scatter(mass_range_logT[:,:220],
                           mass_range_logL[:,:220],
                           c=vrot[:,:220],
                           s=30, cmap="viridis", vmin=125)
            cb = fig.colorbar(vs, ax=ax)
            cb.ax.tick_params(which="major", labelsize=13)
            if j == 2:
                cb.set_label(r"$v_{\rm rot}$ (km s$^{-1}$)", size=17)


        # Find model with most similar mass.
        closest_idx = np.argmin(abs(model_mass-masses))
        closest_model = interped_dfs[closest_idx]
        closest_mass = masses[closest_idx]
        ax.plot(np.log10(closest_model[:,8]),
                np.log10(closest_model[:,6]/ 3.846e33),
                c="k",#"chartreuse",# ls=linestyles[j-2],
                label=fr"{closest_mass:.2f} $M_\odot$ BoOST track")
        
        if j == 1:
            ################## AGE ESTIMATE
            closest_point = np.argmin(np.sqrt((np.log10(closest_model[:,8]) - model_logT)**2 + (np.log10(closest_model[:,6]/ 3.846e33) - model_L)**2))
            within_1sig = (np.log10(closest_model[:,6]/ 3.846e33) < model_L + model_dL) * (np.log10(closest_model[:,6]/ 3.846e33) > model_L - model_dL) & \
                          (np.log10(closest_model[:,8]) < model_logT + model_dlogT) * (np.log10(closest_model[:,8]) > model_logT -  model_dlogT)
            time = (closest_model[:,1])/(3.154e7 * 1e6) # Myr
            age = time[closest_point]
            age_up = time[within_1sig][-1]- age
            age_low = age - time[within_1sig][0]
            if age_low < 0:
                age_low=0
            
            agelabelup = round_to_2(age, age_up)
            agelabellow = round_to_2(age, age_low)
            ax.plot(np.log10(closest_model[:,8])[within_1sig],
                np.log10(closest_model[:,6]/ 3.846e33)[within_1sig],
                "r-", lw=3)
            if i == 0:
                ax.text(0.97, 0.05, fr"Age = {agelabelup[0][:-1]}$^{{+{agelabelup[1][:-1]}}} _{{-{agelabellow[1]}}}$ Myr",
                       transform=ax.transAxes, va="bottom", ha="right", size=14)
        if i == 0:
            ax.set_title(fr"[C] = {abundances[j]} C$_\odot$",
                        size=18, y=1.02)
        ax.legend(markerscale=0.8, fontsize="large")

        xmin, xmax = np.log10(current_row["Teff(K)"]).min()-0.1, np.log10(current_row["Teff(K)"]).max()+0.1
        ymin, ymax = current_row["log(L/Lsun)"].min()-0.2, current_row["log(L/Lsun)"].max()+0.2

        ax.set_xlim(xmin,xmax)
        ax.set_ylim(ymin, ymax)
        if i == 2:
            ax.set_xlabel(r"$\log(T_{\rm eff}/{\rm K})$", size=18)
        if j == 0:
            ax.set_ylabel(r"$\log(L/L_{\odot})$", size=18)
        ax.tick_params(which="major", top=True, right=True, direction="inout", length=8,labelsize=15)
        ax.invert_xaxis()
    plt.tight_layout()
plt.show()
